In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
year = 2022
month = 2
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
df = read_data(input_file)

In [ ]:
# Q2: Add index to records
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
print(y_pred.std())